In [ ]:
using DifferentialEquations
using PyPlot

In [ ]:
cd("../rms")
include("Parse.jl")
include("Reactor.jl")

In [ ]:
phaseDict = readinput("testing/liquid_phase.yml") #load mechanism dictionary

In [ ]:
spcs = phaseDict["phase"]["Species"]; #mechanism dictionaries index:  phaseDict[phasename]["Species" or "Reactions"]
rxns = phaseDict["phase"]["Reactions"];

In [ ]:
solv = Solvent("octane",RiedelViscosity(-98.805,3905.5,14.103,-2.5112e-5,2.0))

In [ ]:
liq = IdealDiluteSolution(spcs,rxns,solv;name="phase",diffusionlimited=true) #Define the phase (how species thermodynamic and kinetic properties calculated)
initialconds = Dict(["T"=>450.0,"P"=>1e5,"V"=>1.0e-6*1e6,"octane"=>6.154e-3*1e6,"oxygen"=>4.953e-6*1e6]) #Set simulation Initial Temp and Pressure
state = MolarState(initialconds,liq) #Define the initial state of the system
domain = ConstantTVDomain(state=state,phase=liq,constantspecies=["oxygen"]) #Define the domain (encodes how system thermodynamic properties calculated)

In [ ]:
react = BatchSingleDomainReactor(domain,(0.0,140000.01)) #Create the reactor object

In [ ]:
sol = solve(react.ode,CVODE_BDF(),abstol=1e-20,reltol=1e-8); #solve the ode associated with the reactor

In [ ]:
ts = exp.(range(log(1e-15),length=1000,stop=log(140000.0)))
xs = [sol(t)./sum(sol(t)) for t in ts]
xmat = hcat(xs...)
maxes = [maximum(xmat[i,:]) for i = 1:size(xmat)[1]]
thresh = 0.01
spnames = []
for i = 1:length(maxes)
    if maxes[i] > thresh
        plot(ts,xmat[i,:])
        push!(spnames,liq.species[i].name)
    end
end
legend(spnames)